In [83]:
#Импорт зависимостей 
import pandas as pd
import numpy as np  
from scipy import stats
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import os
import webbrowser
from plotly.subplots import make_subplots
from pathlib import Path

In [53]:
# чтение csv файла
df = pd.read_csv('ecom_data_prod_1224_0325_upd.csv')

In [84]:
# преобразование строки даты в объект даты
df['researchdate'] = pd.to_datetime(df['researchdate'])
# Создаем папку (если не существует)
visualizations_dir = Path('visualizations')
visualizations_dir.mkdir(exist_ok=True)

In [55]:
df.head()

,SubjectID,QueryText,BrandID,Category1ID,Category2ID,Category3ID,Brand,Category1,Category2,Category3,...,Work,Income,Weight,week_weight,month_weight,Start,researchdate,week,Month,inDelivery
0,3683953663378096962,крем корега,230130,3,303,3032,корега,Лекарства,Личная гигиена,Уход за полостью рта,...,не работает; 12-17; 65+ лет,затрудняюсь ответить,9010.477,9608.837,8405.850,2025-02-04T03:36:38.000+03:00,2025-02-04,2025-02-03,2025-02-01,1
1,432351254888982538,мираторг корм для кошек,232457,19,1901,190199,мираторг,Зоотовары,Корм для кошек,Другое,...,работает,"хватает на еду и одежду, но не на дорогие вещи",5083.769,5089.517,5051.670,2025-02-20T22:09:32.000+03:00,2025-02-20,2025-02-17,2025-02-01,1
2,432351254888982538,корм для кошек мираторг,232457,19,1901,190199,мираторг,Зоотовары,Корм для кошек,Другое,...,работает,"хватает на еду и одежду, но не на дорогие вещи",5254.380,5273.429,5051.670,2025-02-28T20:17:39.000+03:00,2025-02-28,2025-02-24,2025-02-01,1
3,2738205780782286593,deonica,202979,15,1503,15031,deonica,Красота,Парфюмерия и ароматерапия,Дезодоранты,...,работает,"хватает на еду и одежду, но не на дорогие вещи",777.135,811.353,794.963,2025-02-18T18:34:50.000+03:00,2025-02-18,2025-02-17,2025-02-01,1
4,6701361217345430535,глис кур,206222,15,1501,15011,глисс кур,Красота,Уходовая косметика,Средства для волос,...,не работает; 12-17; 65+ лет,хватает на все или почти все,6370.446,5362.518,3920.441,2025-02-12T12:40:13.000+03:00,2025-02-12,2025-02-10,2025-02-01,1


In [56]:
df.columns

Index(['SubjectID', 'QueryText', 'BrandID', 'Category1ID', 'Category2ID',
       'Category3ID', 'Brand', 'Category1', 'Category2', 'Category3',
       'CategoryDelivery', 'ResourceName', 'ResourceType', 'UseType',
       'Platform', 'Sex', 'Age', 'Region', 'FO', 'CountChild', 'Work',
       'Income', 'Weight', 'week_weight', 'month_weight', 'Start',
       'researchdate', 'week', 'Month', 'inDelivery'],
      dtype='object')

In [57]:
groups = df.groupby(['SubjectID','researchdate', 'Brand'])

In [58]:
# формирование статистики по активности респондента за день по бренду: сколько респондент сделал запросов к бренду за день и итоговый вес респондента
daily_brand_user_stats = df.groupby(['researchdate', 'Brand', 'SubjectID']).agg(
    query_count=('QueryText', 'count'), 
    weight=('Weight', 'first')           
).reset_index()

In [59]:
daily_brand_user_stats

,researchdate,Brand,SubjectID,query_count,weight
0,2024-12-01,911 ваша служба спасения,4611694214010660315,1,9373.386
1,2024-12-01,abc (бытовая химия),3386714602605567539,1,1720.954
2,2024-12-01,abc (бытовая химия),6629304165209877379,1,2029.822
3,2024-12-01,abdl,1729390231323622656,1,11903.438
4,2024-12-01,abricot,1585271561336621756,1,1195.283
...,...,...,...,...,...
156610,2025-03-31,яндекс,5773626760334392643,1,2292.826
156611,2025-03-31,яндекс,6052843423965704539,3,3168.700
156612,2025-03-31,яндекс,6629304165209878316,1,2381.889
156613,2025-03-31,яндекс,7575066561518644354,1,2541.140


In [60]:
daily_brand_user_stats['OTS'] = daily_brand_user_stats['weight'] * daily_brand_user_stats['query_count']

In [61]:
daily_brand_user_stats

,researchdate,Brand,SubjectID,query_count,weight,OTS
0,2024-12-01,911 ваша служба спасения,4611694214010660315,1,9373.386,9373.386
1,2024-12-01,abc (бытовая химия),3386714602605567539,1,1720.954,1720.954
2,2024-12-01,abc (бытовая химия),6629304165209877379,1,2029.822,2029.822
3,2024-12-01,abdl,1729390231323622656,1,11903.438,11903.438
4,2024-12-01,abricot,1585271561336621756,1,1195.283,1195.283
...,...,...,...,...,...,...
156610,2025-03-31,яндекс,5773626760334392643,1,2292.826,2292.826
156611,2025-03-31,яндекс,6052843423965704539,3,3168.700,9506.100
156612,2025-03-31,яндекс,6629304165209878316,1,2381.889,2381.889
156613,2025-03-31,яндекс,7575066561518644354,1,2541.140,2541.140


In [62]:
# оставляем только те бренды, что имеют более 6 респондентов и месячный вес более 20000 
significant_brands = daily_brand_user_stats.groupby(['Brand', pd.Grouper(key='researchdate', freq='M')]) \
                     .agg(total_ots=('OTS', 'sum'),
                          unique_users=('SubjectID', 'nunique')) \
                     .query('total_ots >= 20000 and unique_users >= 6') \
                     .reset_index()['Brand'].unique()

C:\Users\Admin\AppData\Local\Temp\ipykernel_16220\879068956.py:2: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [63]:
#Поиск "аномальных" респондентов 

#массив для "аномальных" респондентов 
anomalies = []

# цикл для значимых по условию брендов 
for significant_brand  in significant_brands:
    # значимый бренд
    brand_data = daily_brand_user_stats[daily_brand_user_stats['Brand'] == significant_brand]

    #формируем дневную активность респондентов по бренду 
    for date, day_activity in brand_data.groupby('researchdate'):
        # логарифмируем шкалу для приближения формы распределения к нормальному 
        log_ots = np.log1p(day_activity['OTS'])
        # совершаем z-оценку дневной активности респондентов 
        z_scores = np.abs(stats.zscore(log_ots))
        # оставляем только аномальную активность, выходящую за критический порог 
        date_anomalies = day_activity[z_scores > 2.5]
        # добавляем респондентов с "аномальной" активностью в массив
        anomalies.extend(date_anomalies[['SubjectID', 'researchdate', 'Brand', 'OTS']].values.tolist())

In [64]:
# формируем dataframe для выгрузки в csv 
anomalies_df = pd.DataFrame(anomalies, columns=['SubjectID', 'researchdate', 'Brand', 'OTS'])

In [65]:
# итоговый OTS по дням до очистки
total_daily_ots_before = daily_brand_user_stats.groupby('researchdate').agg(total_ots=('OTS', 'sum')).reset_index()
total_daily_ots_before

,researchdate,total_ots
0,2024-12-01,1.007924e+07
1,2024-12-02,9.985770e+06
2,2024-12-03,1.038658e+07
3,2024-12-04,7.940824e+06
4,2024-12-05,8.960040e+06
...,...,...
116,2025-03-27,9.370695e+06
117,2025-03-28,8.211299e+06
118,2025-03-29,8.710024e+06
119,2025-03-30,9.279740e+06


In [66]:
# удаление активности "аномальных" респондентов, за весь тот день, в который они признаны "аномальными" 
anomalous_days_users = anomalies_df[['researchdate', 'SubjectID']].drop_duplicates()

# маска для "аномальных респондентов"
mask = df.merge(anomalous_days_users, 
                on=['researchdate', 'SubjectID'], 
                how='left', 
                indicator=True)['_merge'] == 'left_only'

cleaned_df = df[mask].copy()
# выгрузка в csv 
cleaned_df.to_csv('cleaned_data.csv', index=False)

In [67]:
# формирование статистики по активности респондента за день по бренду в данных без аномалий 
daily_brand_user_stats_clean = cleaned_df.groupby(['researchdate', 'Brand', 'SubjectID']).agg(
    query_count=('QueryText', 'count'), 
    weight=('Weight', 'first')           
).reset_index()

In [68]:
daily_brand_user_stats_clean['OTS'] = daily_brand_user_stats_clean['weight'] * daily_brand_user_stats_clean['query_count']
daily_brand_user_stats_clean

,researchdate,Brand,SubjectID,query_count,weight,OTS
0,2024-12-01,911 ваша служба спасения,4611694214010660315,1,9373.386,9373.386
1,2024-12-01,abc (бытовая химия),3386714602605567539,1,1720.954,1720.954
2,2024-12-01,abc (бытовая химия),6629304165209877379,1,2029.822,2029.822
3,2024-12-01,abdl,1729390231323622656,1,11903.438,11903.438
4,2024-12-01,abricot,1585271561336621756,1,1195.283,1195.283
...,...,...,...,...,...,...
155480,2025-03-31,яндекс,5773626760334392643,1,2292.826,2292.826
155481,2025-03-31,яндекс,6052843423965704539,3,3168.700,9506.100
155482,2025-03-31,яндекс,6629304165209878316,1,2381.889,2381.889
155483,2025-03-31,яндекс,7575066561518644354,1,2541.140,2541.140


In [69]:
# итоговый OTS по дням после очистки 
total_daily_ots_after = daily_brand_user_stats_clean.groupby('researchdate').agg(total_ots=('OTS', 'sum')).reset_index()
total_daily_ots_after

,researchdate,total_ots
0,2024-12-01,9527265.346
1,2024-12-02,9858644.841
2,2024-12-03,9522073.981
3,2024-12-04,7843702.079
4,2024-12-05,8808670.406
...,...,...
116,2025-03-27,8375709.454
117,2025-03-28,7917048.561
118,2025-03-29,8095187.193
119,2025-03-30,8906216.027


In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=total_daily_ots_before['researchdate'],
    y=total_daily_ots_before['total_ots'],
    mode='lines+markers',
    name='До очистки',
    line=dict(color='blue', width=2),
    marker=dict(size=6)
))

fig.add_trace(go.Scatter(
    x=total_daily_ots_after['researchdate'],
    y=total_daily_ots_after['total_ots'],
    mode='lines+markers',
    name='После очистки',
    line=dict(color='red', width=2),
    marker=dict(size=6)
))

fig.update_layout(
    title='Динамика OTS до и после удаления аномалий',
    xaxis_title='Дата',
    yaxis_title='Общий OTS',
    hovermode='x unified',
    template='plotly_white',
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    xaxis=dict(
        tickformat='%d.%m.%Y',
        tickmode='auto',
        nticks=20,
        gridcolor='lightgray'
    ),
    yaxis=dict(
        gridcolor='lightgray'
    )
)

output_path = visualizations_dir / 'ots_comparison_line_chart.html'
fig.write_html(output_path)

webbrowser.open(f'file://{output_path.resolve()}')

График сохранен по пути: C:\Users\Admin\Desktop\python Проекты\pandas\Проекты Python\Python 4 полусеместровый контроль\Последний полусем\visualizations\ots_comparison_line_chart.html


In [71]:
category_ots_before = df.groupby('CategoryDelivery').agg(total_ots=('Weight', 'sum')).reset_index()
category_ots_before

,CategoryDelivery,total_ots
0,Аудиотехника,9.758682e+07
1,Дезодоранты,7.101670e+06
2,Детские одежда и обувь,1.974693e+07
3,Детское питание,1.363468e+07
4,Женская гигиена,3.712560e+06
5,Корм для кошек,2.407820e+07
6,Корм для собак,7.895212e+06
7,Краски для волос,9.948562e+06
8,Крупная техника,7.494563e+07
9,Макияж для глаз,8.473182e+06


In [72]:
category_ots_after = cleaned_df.groupby('CategoryDelivery').agg(total_ots=('Weight', 'sum')).reset_index()
category_ots_after

,CategoryDelivery,total_ots
0,Аудиотехника,9.187257e+07
1,Дезодоранты,6.992328e+06
2,Детские одежда и обувь,1.961770e+07
3,Детское питание,1.362735e+07
4,Женская гигиена,3.690240e+06
5,Корм для кошек,2.398959e+07
6,Корм для собак,7.894133e+06
7,Краски для волос,9.897885e+06
8,Крупная техника,7.304861e+07
9,Макияж для глаз,8.275078e+06


In [73]:
category_ots_diff = category_ots_before['total_ots'] - category_ots_after['total_ots']
category_ots_diff

0     5.714254e+06
1     1.093423e+05
2     1.292275e+05
3     7.328482e+03
4     2.231983e+04
5     8.861058e+04
6     1.079021e+03
7     5.067782e+04
8     1.897011e+06
9     1.981039e+05
10    3.347975e+04
11    2.805069e+04
12    1.896968e+06
13    1.847913e+06
14    8.391426e+05
15    1.115992e+04
16    1.528773e+05
17    2.858213e+06
18    4.485928e+07
19    2.020157e+05
20    2.533605e+05
21    0.000000e+00
22    2.702715e+04
23    6.055939e+05
24    0.000000e+00
25    0.000000e+00
26    1.870046e+04
27    5.218020e+04
28    0.000000e+00
29    2.231983e+04
30    1.188929e+04
Name: total_ots, dtype: float64

In [74]:
category_ots_diff = pd.concat([category_ots_after['CategoryDelivery'], category_ots_diff],  axis=1)

In [75]:
category_ots_diff

,CategoryDelivery,total_ots
0,Аудиотехника,5.714254e+06
1,Дезодоранты,1.093423e+05
2,Детские одежда и обувь,1.292275e+05
3,Детское питание,7.328482e+03
4,Женская гигиена,2.231983e+04
5,Корм для кошек,8.861058e+04
6,Корм для собак,1.079021e+03
7,Краски для волос,5.067782e+04
8,Крупная техника,1.897011e+06
9,Макияж для глаз,1.981039e+05


In [ ]:
category_ots_diff['Pct_Change'] = (category_ots_after['total_ots'] - category_ots_before['total_ots']) / category_ots_before['total_ots'] * 100

fig = go.Figure()

fig.add_trace(go.Bar(
    x=category_ots_diff['CategoryDelivery'],
    y=category_ots_diff['Pct_Change'],
    marker_color='#1f77b4',  
    textposition='outside',
    texttemplate='%{y:.1f}%',
    hovertemplate='<b>%{x}</b><br>Изменение: %{y:.1f}%<extra></extra>'
))

fig.update_layout(
    title='Изменение OTS по категориям (%)',
    xaxis_title='Категория',
    yaxis_title='Изменение, %',
    yaxis=dict(ticksuffix='%'),
    plot_bgcolor='white',
    hovermode='x',
    xaxis=dict(tickangle=45),
    margin=dict(b=100)
)


output_path = visualizations_dir / 'category_pct_change.html'
fig.write_html(output_path)

webbrowser.open(f'file://{output_path.resolve()}')

True

In [ ]:
anomalies_daily_amount = anomalies_df.groupby('researchdate').agg(anomalies_amount=('SubjectID', 'count')).reset_index()
fig = go.Figure()

fig.add_trace(go.Bar(
    x=anomalies_daily_amount['researchdate'],
    y=anomalies_daily_amount['anomalies_amount'],
    marker_color='indianred',  
    hovertemplate='<b>%{x|%d.%m.%Y}</b><br>Аномалий: %{y}<extra></extra>',
    name='Аномальные респонденты'
))

fig.update_layout(
    title='Количество аномальных респондентов по дням',
    xaxis_title='Дата',
    yaxis_title='Количество аномалий',
    plot_bgcolor='white',
    xaxis=dict(
        tickformat='%d.%m',  
        gridcolor='lightgrey'
    ),
    yaxis=dict(
        gridcolor='lightgrey',
        rangemode='tozero'  
    ),
    bargap=0.2,  
    height=500,
    margin=dict(l=50, r=50, b=100, t=80)
)

output_path = visualizations_dir / 'daily_anomalies_histogram.html'
fig.write_html(output_path)

webbrowser.open(f'file://{output_path.resolve()}')

Гистограмма сохранена: C:\Users\Admin\Desktop\python Проекты\pandas\Проекты Python\Python 4 полусеместровый контроль\Последний полусем\daily_anomalies_histogram.html


In [87]:
def create_category_comparison_chart(level='Category1'):

    before = df.groupby(level)['Weight'].sum().reset_index()
    after = cleaned_df.groupby(level)['Weight'].sum().reset_index()
    
    comparison = before.merge(after, on=level, suffixes=('_before', '_after'))
    comparison['diff'] = comparison['Weight_after'] - comparison['Weight_before']
    comparison['pct_change'] = (comparison['diff'] / comparison['Weight_before']) * 100
    comparison = comparison.sort_values('Weight_before', ascending=False)

    fig = make_subplots(rows=1, cols=2, 
                       specs=[[{"type": "bar"}, {"type": "bar"}]],
                       subplot_titles=[f"Суммарный вес ({level})", f"Изменение веса ({level})"])

    fig.add_trace(
        go.Bar(
            x=comparison[level],
            y=comparison['Weight_before'],
            name='До очистки',
            marker_color='#1f77b4',
            hovertemplate='<b>%{x}</b><br>Было: %{y:,.1f}<extra></extra>'
        ),
        row=1, col=1
    )

    fig.add_trace(
        go.Bar(
            x=comparison[level],
            y=comparison['Weight_after'],
            name='После очистки',
            marker_color='#ff7f0e',
            hovertemplate='<b>%{x}</b><br>Стало: %{y:,.1f}<extra></extra>'
        ),
        row=1, col=1
    )

    fig.add_trace(
        go.Bar(
            x=comparison[level],
            y=comparison['pct_change'],
            marker_color=np.where(comparison['pct_change'] >= 0, '#2ca02c', '#d62728'),
            name='Изменение',
            hovertemplate='<b>%{x}</b><br>Изменение: %{y:.1f}%<extra></extra>',
            texttemplate='%{y:.1f}%',
            textposition='outside'
        ),
        row=1, col=2
    )

    fig.update_layout(
        title_text=f'Сравнение весов до и после очистки ({level})',
        barmode='group',
        height=600,
        showlegend=True,
        plot_bgcolor='white',
        margin=dict(b=150, t=80),
        xaxis1=dict(tickangle=45),
        xaxis2=dict(tickangle=45),
        yaxis1=dict(title='Суммарный вес'),
        yaxis2=dict(title='Изменение (%)', ticksuffix='%')
    )

    output_path = visualizations_dir / f'weight_comparison_{level}.html'
    fig.write_html(output_path, auto_open=False)
    webbrowser.open('file://' + os.path.realpath(output_path))
    
    return fig

for level in ['Category1', 'Category2', 'Category3']:
    create_category_comparison_chart(level)